# Day 16: Cell - Building Footprints in Uzbekistan
## 30 Day Map Challenge

This notebook visualizes building footprints from Microsoft Buildings dataset for Tashkent, Uzbekistan, with a grid overlay to represent the 'cell' theme.

### Install required packages (if needed)
```bash
pip install pystac-client planetary-computer geopandas matplotlib contextily folium pyarrow
```

In [ ]:
# Import required libraries
from pystac_client import Client
import planetary_computer as pc
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.colors import LinearSegmentedColormap
import contextily as ctx
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

### Connect to Planetary Computer STAC API

In [ ]:
# Search against the Planetary Computer STAC API
catalog = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1"
)

print("Connected to Planetary Computer STAC API")

### Define Area of Interest (Tashkent, Uzbekistan)

In [ ]:
# Define your area of interest - Tashkent region
from shapely.geometry import shape

aoi = {
    "type": "Polygon",
    "coordinates": [
        [
            [69.10337206279829, 41.23280114907183],
            [69.44412384414161, 41.23280114907183],
            [69.44412384414161, 41.39922765534888],
            [69.10337206279829, 41.39922765534888],
            [69.10337206279829, 41.23280114907183]
        ]
    ]
}

# Convert AOI to GeoDataFrame - more robust method
aoi_geom = shape(aoi)
aoi_gdf = gpd.GeoDataFrame({'geometry': [aoi_geom]}, crs="EPSG:4326")

print(f"AOI defined: {aoi_gdf.bounds}")

### Search for Microsoft Buildings Data

In [ ]:
# Define your search with CQL2 syntax
search = catalog.search(filter_lang="cql2-json", filter={
    "op": "and",
    "args": [
        {"op": "s_intersects", "args": [{"property": "geometry"}, aoi]},
        {"op": "=", "args": [{"property": "collection"}, "ms-buildings"]}
    ]
})

# Grab the first item from the search results and sign the assets
items = list(search.get_items())
print(f"Found {len(items)} item(s) in the search results")

if items:
    first_item = items[0]
    signed_item = pc.sign_item(first_item)
    print(f"\nAvailable assets: {list(signed_item.assets.keys())}")
    
    # Get the data URL
    data_url = signed_item.assets["data"].href
    print(f"\nData URL: {data_url}")

### Load Building Footprints

In [ ]:
# Read the building footprints from the GeoParquet file
print("Loading building footprints... This may take a moment.")

try:
    # Try reading with bbox filter
    buildings = gpd.read_parquet(data_url, bbox=aoi_gdf.total_bounds)
    print(f"Successfully loaded {len(buildings):,} buildings")
except Exception as e:
    print(f"Error loading with bbox: {e}")
    print("Trying alternative loading method...")
    # Try without bbox filter
    buildings = gpd.read_parquet(data_url)
    # Filter manually
    buildings = buildings[buildings.geometry.intersects(aoi_gdf.unary_union)]
    print(f"Loaded and filtered to {len(buildings):,} buildings")

# Ensure CRS is set
if buildings.crs is None:
    buildings = buildings.set_crs("EPSG:4326")

print(f"\nBuildings GeoDataFrame info:")
print(f"Shape: {buildings.shape}")
print(f"CRS: {buildings.crs}")
print(f"Columns: {buildings.columns.tolist()}")
print(f"\nFirst few rows:")
print(buildings.head())

### Calculate Building Statistics

In [ ]:
# Calculate building areas
buildings_projected = buildings.to_crs(epsg=32642)  # UTM Zone 42N for Uzbekistan
buildings_projected['area_sqm'] = buildings_projected.geometry.area

print("Building Statistics:")
print(f"Total buildings: {len(buildings):,}")
print(f"Total area: {buildings_projected['area_sqm'].sum()/1e6:.2f} km²")
print(f"Average building size: {buildings_projected['area_sqm'].mean():.2f} m²")
print(f"Median building size: {buildings_projected['area_sqm'].median():.2f} m²")
print(f"Largest building: {buildings_projected['area_sqm'].max():.2f} m²")
print(f"Smallest building: {buildings_projected['area_sqm'].min():.2f} m²")

### Create Grid Cells for the 'Cell' Theme

In [ ]:
# Create a grid to represent 'cells'
from shapely.geometry import box
import numpy as np

# Get bounds
minx, miny, maxx, maxy = aoi_gdf.total_bounds

# Create grid with specified cell size (in degrees)
cell_size = 0.02  # Approximately 2 km
grid_cells = []
x_coords = np.arange(minx, maxx, cell_size)
y_coords = np.arange(miny, maxy, cell_size)

for x in x_coords:
    for y in y_coords:
        grid_cells.append(box(x, y, x + cell_size, y + cell_size))

grid_gdf = gpd.GeoDataFrame({'geometry': grid_cells}, crs="EPSG:4326")
print(f"Created {len(grid_gdf)} grid cells")

# Calculate building count per cell using spatial join
print("Calculating building statistics per cell...")
try:
    # Method 1: Using projected coordinates
    grid_gdf_projected = grid_gdf.to_crs(epsg=32642)
    buildings_projected_copy = buildings.to_crs(epsg=32642)
    buildings_projected_copy['area_sqm'] = buildings_projected_copy.geometry.area
    
    # Spatial join to count buildings per cell
    joined = gpd.sjoin(grid_gdf_projected, buildings_projected_copy, how='left', predicate='intersects')
    
    # Count buildings per cell
    cell_counts = joined.groupby(joined.index).size()
    grid_gdf_projected['building_count'] = 0  # Initialize with 0
    grid_gdf_projected.loc[cell_counts.index, 'building_count'] = cell_counts.values
    
    # Calculate building area per cell
    building_area_per_cell = joined.groupby(joined.index)['area_sqm'].sum()
    grid_gdf_projected['building_area'] = 0  # Initialize with 0
    grid_gdf_projected.loc[building_area_per_cell.index, 'building_area'] = building_area_per_cell.values
    
    # Calculate density
    grid_gdf_projected['cell_area'] = grid_gdf_projected.geometry.area
    grid_gdf_projected['density'] = (grid_gdf_projected['building_area'] / grid_gdf_projected['cell_area']) * 100
    
    # Convert back to WGS84 for plotting
    grid_gdf = grid_gdf_projected.to_crs(epsg=4326)
    
    print(f"Building count per cell - Min: {grid_gdf['building_count'].min():.0f}, Max: {grid_gdf['building_count'].max():.0f}, Mean: {grid_gdf['building_count'].mean():.1f}")
    print(f"Successfully calculated grid statistics!")
    
except Exception as e:
    print(f"Error in spatial join: {e}")
    print("Using alternative method...")
    # Fallback: simple spatial join
    grid_gdf['building_count'] = 0
    grid_gdf['density'] = 0.0
    for idx, cell in grid_gdf.iterrows():
        count = buildings[buildings.geometry.intersects(cell.geometry)].shape[0]
        grid_gdf.at[idx, 'building_count'] = count
    print("Grid statistics calculated with fallback method")

### Visualization 1: Building Footprints with Grid Overlay

In [ ]:
# Create the main visualization
fig, ax = plt.subplots(figsize=(20, 16), dpi=150)

# Plot grid cells colored by building density
grid_gdf.plot(
    column='density',
    ax=ax,
    cmap='YlOrRd',
    alpha=0.6,
    edgecolor='white',
    linewidth=0.5,
    legend=True,
    legend_kwds={'label': 'Building Density (%)', 'orientation': 'horizontal', 'shrink': 0.8, 'pad': 0.05}
)

# Plot building footprints
buildings.plot(
    ax=ax,
    facecolor='#2E4057',
    edgecolor='none',
    alpha=0.7,
    linewidth=0.1
)

# Add basemap
try:
    ctx.add_basemap(
        ax,
        crs=buildings.crs.to_string(),
        source=ctx.providers.CartoDB.Positron,
        alpha=0.5
    )
except:
    print("Could not add basemap - continuing without it")

# Styling
ax.set_axis_off()
plt.title(
    'Day 16: Cell - Building Footprints in Tashkent, Uzbekistan\n30 Day Map Challenge',
    fontsize=24,
    fontweight='bold',
    pad=20
)

# Add text annotation
info_text = f"Total Buildings: {len(buildings):,}\nGrid Cells: {len(grid_gdf)}\nData: Microsoft Building Footprints"
plt.text(
    0.02, 0.98, info_text,
    transform=ax.transAxes,
    fontsize=12,
    verticalalignment='top',
    bbox=dict(boxstyle='round', facecolor='white', alpha=0.8)
)

plt.tight_layout()
plt.savefig('Day16_Cell_Buildings.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

print("Map saved as 'Day16_Cell_Buildings.png'")

### Visualization 2: Building Count Heatmap by Cell

In [ ]:
# Create heatmap visualization
fig, ax = plt.subplots(figsize=(20, 16), dpi=150)

# Plot grid cells colored by building count
grid_gdf.plot(
    column='building_count',
    ax=ax,
    cmap='viridis',
    alpha=0.8,
    edgecolor='white',
    linewidth=1,
    legend=True,
    legend_kwds={'label': 'Building Count per Cell', 'orientation': 'horizontal', 'shrink': 0.8, 'pad': 0.05}
)

# Add basemap
try:
    ctx.add_basemap(
        ax,
        crs=buildings.crs.to_string(),
        source=ctx.providers.CartoDB.Positron,
        alpha=0.3
    )
except:
    print("Could not add basemap - continuing without it")

# Styling
ax.set_axis_off()
plt.title(
    'Building Density Heatmap by Cell\nTashkent, Uzbekistan',
    fontsize=24,
    fontweight='bold',
    pad=20
)

plt.tight_layout()
plt.savefig('Day16_Cell_Heatmap.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

print("Heatmap saved as 'Day16_Cell_Heatmap.png'")

### Visualization 3: Building Size Distribution

In [ ]:
# Create building size distribution plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Histogram of building sizes
buildings_projected['area_sqm'].hist(bins=50, ax=ax1, color='#2E4057', edgecolor='white')
ax1.set_xlabel('Building Area (m²)', fontsize=12)
ax1.set_ylabel('Frequency', fontsize=12)
ax1.set_title('Distribution of Building Sizes', fontsize=14, fontweight='bold')
ax1.set_xlim(0, buildings_projected['area_sqm'].quantile(0.95))  # Show 95% of data
ax1.grid(alpha=0.3)

# Bar chart of building count by cell
top_cells = grid_gdf.nlargest(20, 'building_count')
ax2.barh(range(len(top_cells)), top_cells['building_count'].values, color='#E74C3C')
ax2.set_xlabel('Building Count', fontsize=12)
ax2.set_ylabel('Grid Cell', fontsize=12)
ax2.set_title('Top 20 Cells by Building Count', fontsize=14, fontweight='bold')
ax2.invert_yaxis()
ax2.grid(alpha=0.3, axis='x')

plt.tight_layout()
plt.savefig('Day16_Cell_Statistics.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

print("Statistics plot saved as 'Day16_Cell_Statistics.png'")

### Interactive Map with Folium

In [ ]:
import folium
from folium import plugins
import json

# Calculate center of AOI
center_lat = (miny + maxy) / 2
center_lon = (minx + maxx) / 2

# Create base map
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=12,
    tiles='CartoDB positron'
)

# Prepare grid data for Folium - add index as a column for proper key mapping
grid_for_map = grid_gdf.copy()
grid_for_map['id'] = grid_for_map.index.astype(str)
grid_for_map['building_count_val'] = grid_for_map['building_count']

# Add grid cells with color based on building count
try:
    folium.Choropleth(
        geo_data=grid_for_map.to_json(),
        data=grid_for_map,
        columns=['id', 'building_count_val'],
        key_on='feature.properties.id',
        fill_color='YlOrRd',
        fill_opacity=0.6,
        line_opacity=0.8,
        legend_name='Building Count per Cell'
    ).add_to(m)
except Exception as e:
    print(f"Could not add choropleth: {e}")
    # Fallback: just add the grid as GeoJson
    folium.GeoJson(
        grid_for_map,
        name='Grid Cells',
        style_function=lambda x: {
            'fillColor': '#ffaf00',
            'color': 'white',
            'weight': 1,
            'fillOpacity': 0.4
        }
    ).add_to(m)

# Add building footprints (sample for performance)
if len(buildings) > 1000:
    buildings_sample = buildings.sample(n=1000, random_state=42)
    print(f"Sampling {len(buildings_sample)} buildings for interactive map (out of {len(buildings):,} total)")
else:
    buildings_sample = buildings
    print(f"Adding all {len(buildings_sample)} buildings to interactive map")

folium.GeoJson(
    buildings_sample.__geo_interface__,
    name='Buildings (sample)',
    style_function=lambda x: {
        'fillColor': '#2E4057',
        'color': '#2E4057',
        'weight': 0.5,
        'fillOpacity': 0.7
    }
).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Add title
title_html = '''
             <div style="position: fixed; 
                         top: 10px; left: 50px; width: 400px; height: 90px; 
                         background-color:white; border:2px solid grey; z-index:9999; 
                         font-size:16px; padding: 10px">
             <h4>Day 16: Cell - Building Footprints</h4>
             <p>Tashkent, Uzbekistan<br>
             Buildings: ''' + f"{len(buildings):,}" + '''</p>
             </div>
             '''
m.get_root().html.add_child(folium.Element(title_html))

# Save map
m.save('Day16_Cell_Interactive.html')
print("Interactive map saved as 'Day16_Cell_Interactive.html'")

# Display in notebook
m

In [ ]:
# Load Tashkent boundary and clip buildings
tashkent = gpd.read_file('Boundaries/Tashkent_boundaries.geojson')
print(f"Loaded Tashkent boundary")

# Clip buildings to Tashkent boundary
print(f"Buildings before clipping: {len(buildings):,}")
buildings_tashkent = gpd.clip(buildings, tashkent)
print(f"Buildings after clipping to Tashkent: {len(buildings_tashkent):,}")

# Create grid cells
from shapely.geometry import box
import numpy as np

minx, miny, maxx, maxy = buildings_tashkent.total_bounds
cell_size = 0.01  # ~1km cells

x_coords = np.arange(minx, maxx + cell_size, cell_size)
y_coords = np.arange(miny, maxy + cell_size, cell_size)

grid_cells = []
for x in x_coords[:-1]:
    for y in y_coords[:-1]:
        grid_cells.append(box(x, y, x + cell_size, y + cell_size))

grid = gpd.GeoDataFrame({'geometry': grid_cells}, crs="EPSG:4326")
grid = gpd.clip(grid, tashkent)

# Count buildings per cell
print("Counting buildings per cell...")
grid['building_count'] = 0
for idx, cell in grid.iterrows():
    count = buildings_tashkent[buildings_tashkent.geometry.intersects(cell.geometry)].shape[0]
    grid.at[idx, 'building_count'] = count

print(f"Created {len(grid)} grid cells")
print(f"Building count range: {grid['building_count'].min():.0f} - {grid['building_count'].max():.0f}")

In [ ]:
# Create Folium map with buildings and grid cells
import folium

center = [tashkent.geometry.centroid.y.iloc[0], tashkent.geometry.centroid.x.iloc[0]]

m = folium.Map(location=center, zoom_start=11, tiles='CartoDB positron')

# Add Tashkent boundary
folium.GeoJson(
    tashkent,
    name='Tashkent Boundary',
    style_function=lambda x: {
        'fillColor': 'none',
        'color': 'red',
        'weight': 3,
        'fillOpacity': 0
    }
).add_to(m)

# Color function for grid cells
max_count = grid['building_count'].max()

def get_color(count):
    if count == 0:
        return '#f0f0f0'
    elif count < max_count * 0.2:
        return '#fee5d9'
    elif count < max_count * 0.4:
        return '#fcae91'
    elif count < max_count * 0.6:
        return '#fb6a4a'
    elif count < max_count * 0.8:
        return '#de2d26'
    else:
        return '#a50f15'

# Add grid cells with tooltips
for idx, row in grid.iterrows():
    folium.GeoJson(
        row.geometry,
        style_function=lambda x, count=row['building_count']: {
            'fillColor': get_color(count),
            'color': 'white',
            'weight': 0.5,
            'fillOpacity': 0.7
        },
        tooltip=f\"Buildings: {int(row['building_count'])}\"
    ).add_to(m)

# Sample buildings for display
if len(buildings_tashkent) > 5000:
    buildings_sample = buildings_tashkent.sample(n=5000, random_state=42)
    print(f\"Displaying 5,000 sample buildings (total: {len(buildings_tashkent):,})\")\nelse:
    buildings_sample = buildings_tashkent
    print(f\"Displaying all {len(buildings_tashkent):,} buildings\")

# Add buildings
folium.GeoJson(
    buildings_sample,
    name='Buildings',
    style_function=lambda x: {
        'fillColor': '#2E4057',
        'color': '#2E4057',
        'weight': 0.5,
        'fillOpacity': 0.7
    }
).add_to(m)

# Add title
title_html = f'''
<div style=\"position: fixed; 
            top: 10px; left: 50px; width: 450px; height: 110px; 
            background-color:white; border:2px solid grey; z-index:9999; 
            font-size:14px; padding: 10px\">
    <h4 style=\"margin:0; padding-bottom:5px;\">Day 16: Cell - Building Footprints</h4>
    <p style=\"margin:5px 0;\"><strong>Location:</strong> Tashkent, Uzbekistan</p>
    <p style=\"margin:5px 0;\"><strong>Buildings:</strong> {len(buildings_tashkent):,}</p>
    <p style=\"margin:5px 0;\"><strong>Grid Cells:</strong> {len(grid)}</p>
</div>
'''
m.get_root().html.add_child(folium.Element(title_html))

folium.LayerControl().add_to(m)

# Save and display
m.save('Day16_Cell_Tashkent.html')
print(\"\\nMap saved as 'Day16_Cell_Tashkent.html'\")

m"

### Summary

This notebook demonstrates:
1. Accessing Microsoft Building Footprints via Planetary Computer
2. Creating a grid overlay to represent 'cells' matching the challenge theme
3. Calculating building statistics and density metrics per cell
4. Creating multiple visualizations including static maps and interactive HTML map
5. Analyzing spatial distribution of buildings across Tashkent, Uzbekistan

**Data Source:** Microsoft Building Footprints via Planetary Computer  
**Challenge:** 30 Day Map Challenge - Day 16: Cell  
**Location:** Tashkent, Uzbekistan

### Clip Buildings to Tashkent Boundary and Create Grid

In [ ]:
# Load Tashkent boundary
tashkent = gpd.read_file('Boundaries/Tashkent_boundaries.geojson')
print(f"Loaded Tashkent boundary")

# Clip buildings to Tashkent boundary
print(f"Buildings before clipping: {len(buildings):,}")
buildings_tashkent = gpd.clip(buildings, tashkent)
print(f"Buildings after clipping to Tashkent: {len(buildings_tashkent):,}")

# Create grid cells
from shapely.geometry import box
import numpy as np

minx, miny, maxx, maxy = buildings_tashkent.total_bounds
cell_size = 0.01  # ~1km cells

x_coords = np.arange(minx, maxx + cell_size, cell_size)
y_coords = np.arange(miny, maxy + cell_size, cell_size)

grid_cells = []
for x in x_coords[:-1]:
    for y in y_coords[:-1]:
        grid_cells.append(box(x, y, x + cell_size, y + cell_size))

grid = gpd.GeoDataFrame({'geometry': grid_cells}, crs="EPSG:4326")
grid = gpd.clip(grid, tashkent)

# Count buildings per cell
print("Counting buildings per cell...")
grid['building_count'] = 0
for idx, cell in grid.iterrows():
    count = buildings_tashkent[buildings_tashkent.geometry.intersects(cell.geometry)].shape[0]
    grid.at[idx, 'building_count'] = count

print(f"Created {len(grid)} grid cells")
print(f"Building count range: {grid['building_count'].min():.0f} - {grid['building_count'].max():.0f}")

### Final Folium Map with Tashkent Boundary